In [39]:
'''
In this file, we compose density matrices with each other. 

We only investigate such situations where a pure density matrices is composed with a mixed one. 

'''
from qiskit import QuantumCircuit
import qiskit.quantum_info as qi

import numpy as np

In [40]:
'''
consider the maximally mixed state, with entropy one
'''
qc = QuantumCircuit(2)
qc.h(1)
qc.cnot(1,0)

tempDM = qi.DensityMatrix(qc)
maximallyMixedDM = qi.partial_trace(tempDM, [0])

/tmp/ipykernel_4975/2081203963.py:6: DeprecationWarning: The method ``qiskit.circuit.quantumcircuit.QuantumCircuit.cnot()`` is deprecated as of qiskit 0.45.0. It will be removed no earlier than 3 months after the release date. Use QuantumCircuit.cx as direct replacement.
  qc.cnot(1,0)


In [41]:
'''
Now, we create the pure density matrix corresponding to some other noun. 
'''

def DensityMatrixFromWord(weights):
    '''
    we start with only nouns: 
    '''
    qc = QuantumCircuit(1)

    qc.h(0)
    # apply the three gates from the IQP ansatz
    qc.rx(weights[0], 0)
    qc.rz(weights[1], 0)
    qc.rx(weights[2], 0)
    qc.h(0)

    state = qi.Statevector.from_label('0')
    dM = qi.DensityMatrix(state)
    newDM = dM.evolve(qc)

    return newDM


In [42]:
def SpectralDecomposition(rho):

    eigenvalues, eigenvectors = np.linalg.eig(rho)
    # rhoDiagonalized = np.diag(eigenvalues)

    normalizedEvs = []
    for eV in eigenvectors: 
        normalizedEvs.append(eV / np.linalg.norm(eV))


    projectors = []

    for eV in normalizedEvs: 
        projectors.append(np.outer((eV), eV))

    return eigenvalues, projectors




def SpiderAnd(rho, sigma): 
    '''
    ...
    '''
    return np.matmul(rho, sigma)

def Fuzz(rho, sigma): 
    '''
    ...
    '''
    eVList, projectors = SpectralDecomposition(sigma)

    print('testetsetse')
    print(projectors[0] * projectors[0])

    termOne = eVList[0] * ( np.matmul(np.matmul(projectors[0], rho), projectors[0])) 
    termTwo = eVList[1] * ( np.matmul(np.matmul(projectors[1], rho), projectors[1]))  

    return termOne + termTwo


def Phaser(rho, sigma): 
    '''
    ...
    '''
    return np.matmul(np.sqrt(sigma), np.matmul(rho, np.sqrt(sigma)))

In [43]:
'''
import the weights
'''
import pickle 

with open('../weights/weightsDictTK1QB_gijs_500.pkl', 'rb') as fp: 
    parameterDictTK = pickle.load(fp)



chefWeights = []
for i in parameterDictTK: 
    if 'chef' in i and not '†' in i: 
        chefWeights.append(parameterDictTK[i])

programmerWeights = []
for i in parameterDictTK: 
    if 'programmer' in i and not '†' in i: 
        programmerWeights.append(parameterDictTK[i])

In [44]:
def CompareMethods():
    '''
    
    '''
    dm1 = maximallyMixedDM

    # some word, let's say chef
    dm2 = DensityMatrixFromWord(programmerWeights)

    # spider
    spiderResult = SpiderAnd(dm1, dm2)


    # fuzz 
    fuzzResult = Fuzz(dm1, dm2)


    # phaser 
    phaserResult = Phaser(dm1, dm2)


    print(np.trace(spiderResult))
    print(np.trace(fuzzResult))
    print(np.trace(phaserResult))

    # normalizing
    spiderResult = spiderResult / np.trace(spiderResult)
    fuzzResult = fuzzResult / np.trace(fuzzResult)
    phaserResult = phaserResult / np.trace(phaserResult)

    print(np.trace(spiderResult))
    print(np.trace(fuzzResult))
    print(np.trace(phaserResult))


    # to compare to actual label
    iTMatrix = np.array([[0,0], [0,1]])
    foodMatrix = np.array([[1,0], [0,0]])

    iTDM = qi.DensityMatrix(iTMatrix)
    foodDM = qi.DensityMatrix(foodMatrix)





    print('spider: ')

    print(qi.state_fidelity(spiderResult, foodMatrix, validate = False))

    print('fuzz: ', qi.state_fidelity(fuzzResult, foodMatrix, validate = False))

    print('phaser', qi.state_fidelity(phaserResult, foodMatrix, validate = False))

CompareMethods()


testetsetse
[[ 0.90911679+0.j         -0.04332435+0.00952652j]
 [-0.04332435+0.00952652j  0.00196481-0.00090798j]]
(0.4999999999999996+0j)
(0.41221645113992517+0.009072525508944891j)
(0.7106166456403074+4.163336342344337e-17j)
(1+0j)
(1+3.469446951953614e-18j)
(0.9999999999999999+4.622231866529366e-33j)
spider: 
0.9534761609822778
fuzz:  1.0499765526553086
phaser 0.8190722900767886
